In [28]:
import torch
import numpy as np
from torch import nn
from torchvision import transforms

# EfficientNet-B0 model

In [15]:
from efficientnet_pytorch import EfficientNet
class MyModel(nn.Module):
    def __init__(self, model_name: str):
        super(MyModel,self).__init__()

        # self.model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)
        self.model = EfficientNet.from_pretrained(model_name)
        self.model.set_swish(memory_efficient=False)
        self.linear1 = torch.nn.Linear(1000, 1)
        self.activation3 = torch.nn.Sigmoid()

    def forward(self,x):
        answ = self.model(x)
        #print(answ.shape)
        #answ = self.encoder(answ)
        #print(answ.shape)
        answ = self.activation3(self.linear1(answ))

        return answ

In [16]:
transform_efnet=transforms.Compose([transforms.Resize((150,150)),
                              transforms.ToTensor()
                              ])

In [17]:
def init_ef_model(model_name, model_path):
    device = torch.device('cpu')
    model = MyModel(model_name)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval();
    return model

In [18]:
def get_res_efnet(image_path: str, model: MyModel):
    image = Image.open(image_path)
    image = transform_efnet(image).unsqueeze(0)

    # model.cuda()
    # Set layers such as dropout and batchnorm in evaluation mode
    # model.eval();

    # Get the 1000-dimensional model output
    with torch.no_grad():
        out = model(image)
    # print(out.detach().numpy()[0][0])    
    return out.detach().numpy()[0][0]

In [19]:
dummy_input = torch.randn(1, 3, 150, 150, device='cpu')

In [20]:
model_b0_name = 'efficientnet-b0'
model_b0_path = 'PretrainedModels/GenderEstimation/Ours/ef_b0.pth'
b0_model = init_ef_model(model_b0_name, model_b0_path)

Loaded pretrained weights for efficientnet-b0


# Export model to ONNX format

In [23]:
torch.onnx.export(b0_model, dummy_input, "efnet_b0.onnx")

# Check model

In [25]:
import onnx

# Load the ONNX model
model = onnx.load("efnet_b0.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
# print(onnx.helper.printable_graph(model.graph))

# Usage onnx model and checking results

In [51]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("efnet_b0.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_outs = ort_session.run(None, ort_inputs)

# compute pytorch prediction
torch_out = b0_model(dummy_input)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)
print("Exported model has been tested with ONNXRuntime, and the result looks good!")

torch_res = torch_out.detach().numpy()[0][0]
onnx_res = ort_outs[0][0][0]
print("Torch prediction: %0.5f" % torch_res + "\nONNX prediction: %0.5f" % onnx_res)

Exported model has been tested with ONNXRuntime, and the result looks good!
Torch prediction: 0.94459
ONNX prediction: 0.94459
